In [1]:
import pandas as pd 
import numpy as np 
import torch 
import ast
import matplotlib.pyplot as plt

In [ ]:
train_dict = torch.load("data/train_pv_xrd.pt")
val_dict = torch.load("data/val_pv_xrd.pt")
#test_dict = torch.load("test_pv_xrd.pt")

In [ ]:
train_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/val.csv")
#test_df = pd.read_csv("test.csv")

In [ ]:

def df_to_spacegroup_data(df): 
    spacegroup_number = np.array(df['spacegroup.number'])
    spacegroup_number_one_hot_encoded = np.eye(230)[spacegroup_number]
    
    return torch.tensor(spacegroup_number_one_hot_encoded)

In [ ]:
data_dict = {
    'train': (train_dict, train_df),
    'val': (val_dict, val_df), 
#    'test': (test_dict, test_df)
}

In [ ]:
def df_to_spacegroup_data(df): 
    spacegroup_number = np.array(df['spacegroup.number'])
    spacegroup_number_one_hot_encoded = np.eye(230)[spacegroup_number]
    
    return torch.tensor(spacegroup_number_one_hot_encoded)

In [ ]:
training_sgs = torch.tensor(train_df['spacegroup.number'], dtype=torch.long)
val_sgs = torch.tensor(val_df['spacegroup.number'], dtype=torch.long)
#test_sgs = df_to_spacegroup_data(test_df)

In [ ]:
def convert_to_tensor(pseudo_voight_dict, df): 
    list_of_pseudo_voights = []
    for key in df['material_id']:
        key += "_0"
        value = pseudo_voight_dict[key]
        list_of_pseudo_voights.append(value)

    tensor_of_pseudo_voights = torch.tensor(torch.stack(list_of_pseudo_voights))
    return(tensor_of_pseudo_voights)

In [ ]:
training_pvs = convert_to_tensor(train_dict, train_df)
val_pvs = convert_to_tensor(val_dict, val_df)
#test_pvs = convert_to_tensor(test_dict)

In [ ]:
train_df['atomic_numbers'] = train_df['atomic_numbers'].apply(ast.literal_eval)
val_df['atomic_numbers'] = val_df['atomic_numbers'].apply(ast.literal_eval)

In [ ]:
from collections import Counter

lst = [4, 4, 5, 6]
count_dict = Counter(lst)

In [ ]:
def create_multi_hot_embedding(input_list, max_value = 99):
    # Initialize an array of zeros with length max_value + 1
    multi_hot_embedding = np.zeros(max_value + 1, dtype=int)
    
    #subtract out 1 to index from 0 (hydrogen)
    input_list = [value - 1 for value in input_list]

    # Set the corresponding index for each number in the input list to 1
    for num in input_list:
        multi_hot_embedding[num] += 1
    
    return multi_hot_embedding

In [ ]:
def create_2D_embedding(input_list, max_value = 99):
    # Initialize an array of zeros with length max_value + 1
    multi_hot_embedding = np.zeros((21, max_value + 1), dtype=int)
    
    #subtract out 1 to index from 0 (hydrogen)
    input_list = [value - 1 for value in input_list]
    count_dict = Counter(input_list)

    # Set the corresponding index for each number in the input list to 1
    for key, value in count_dict.items():
        multi_hot_embedding[value, key] = 1
    
    return multi_hot_embedding

In [ ]:
train_df['composition_multihot'] = train_df['atomic_numbers'].apply(create_multi_hot_embedding)
val_df['composition_multihot'] = val_df['atomic_numbers'].apply(create_multi_hot_embedding)

train_df['composition_2D'] = train_df['atomic_numbers'].apply(create_2D_embedding)
val_df['composition_2D'] = val_df['atomic_numbers'].apply(create_2D_embedding)


In [ ]:
train_composition = torch.tensor(train_df['composition_multihot'])
val_composition = torch.tensor(val_df['composition_multihot'])

train_composition = train_composition.unsqueeze(1)
val_composition = val_composition.unsqueeze(1)

In [ ]:
train_composition2D = torch.tensor(train_df['composition_2D'])
val_composition2D = torch.tensor(val_df['composition_2D'])

train_composition2D = train_composition2D.unsqueeze(1)
val_composition2D = val_composition2D.unsqueeze(1)

In [ ]:
# Create a random tensor with the shape [21, 100]
tensor = train_composition2D[4][0]

# Use matplotlib to create a heatmap
plt.figure(figsize=(10, 5))
plt.imshow(tensor, aspect='auto')
plt.colorbar()
plt.title('Heatmap of Tensor')
plt.xlabel('Column Index')
plt.ylabel('Row Index')
plt.show()

In [ ]:
train_composition = train_composition.float()
val_composition = val_composition.float()

train_composition2D = train_composition2D.float()
val_composition2D = val_composition2D.float()

In [ ]:
torch.save(training_pvs, "data/train_pvs.pt")
torch.save(val_pvs, "data/val_pvs.pt")
torch.save(training_sgs, "data/train_sgs.pt")
torch.save(val_sgs, "data/val_sgs.pt")
torch.save(train_composition, "data/train_composition.pt")
torch.save(val_composition, "data/val_composition.pt")

torch.save(train_composition2D, "data/train_composition2D.pt")
torch.save(val_composition2D, "data/val_composition2D.pt")